In [120]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import hvplot.pandas
%matplotlib inline

In [121]:
fraud_reporting = pd.read_csv(Path("Resources/SARStats2014_2021.csv"), thousands=',', index_col='Year Month', parse_dates=True, infer_datetime_format = True)
fraud_reporting

,Industry,Suspicious Activity,Count
Year Month,,,
2014 January,Depository Institution,ACH,1505
2014 January,Depository Institution,Business Loan,147
2014 January,Depository Institution,Check,8044
2014 January,Depository Institution,Consumer Loan (see instructions),7419
2014 January,Depository Institution,Credit/Debit Card,5572
...,...,...,...
2021 December,Depository Institution,Securities,29
2021 December,Depository Institution,Wire,5914
2021 December,Depository Institution,[Total],93692


In [123]:
fraud_reporting = fraud_reporting.loc[fraud_reporting["Suspicious Activity"]!="[Total]" ]
fraud_reporting = fraud_reporting.drop(columns= 'Suspicious Activity')
fraud_reporting.index = pd.to_datetime(fraud_reporting.index, infer_datetime_format = True)
fraud_reporting

,Industry,Count
Year Month,,
2014-01-01,Depository Institution,1505
2014-01-01,Depository Institution,147
2014-01-01,Depository Institution,8044
2014-01-01,Depository Institution,7419
2014-01-01,Depository Institution,5572
...,...,...
2021-12-01,Depository Institution,16703
2021-12-01,Depository Institution,23
2021-12-01,Depository Institution,12


In [124]:
quarterly_fraud_reporting = fraud_reporting.groupby([fraud_reporting.index.year,fraud_reporting.index.quarter]).sum()
quarterly_fraud_reporting

Count
Year Month Year Month        
2014       1            82364
           2            95516
           3            94084
           4            90868
2015       1            90566
           2            96365
           3           101745
           4           105491
2016       1           108801
           2           128668
           3           126907
           4           115900
2017       1           117009
           2           121347
           3           125990
           4           125829
2018       1           132985
           2           140769
           3           152955
           4           165279
2019       1           186318
           2           169526
           3           177728
           4           185082
2020       1           189439
           2           176864
           3           224264
           4           202986
2021       1           201403
           2           231668
           3           273794
           4           265021

In [125]:
quarterly_fraud_reporting = quarterly_fraud_reporting.rename(columns={'Count':'Fraud Reporting Count'})
quarterly_fraud_reporting.index = quarterly_fraud_reporting.index.rename(['Year','Quarter'])
quarterly_fraud_reporting

Fraud Reporting Count
Year Quarter                       
2014 1                        82364
     2                        95516
     3                        94084
     4                        90868
2015 1                        90566
     2                        96365
     3                       101745
     4                       105491
2016 1                       108801
     2                       128668
     3                       126907
     4                       115900
2017 1                       117009
     2                       121347
     3                       125990
     4                       125829
2018 1                       132985
     2                       140769
     3                       152955
     4                       165279
2019 1                       186318
     2                       169526
     3                       177728
     4                       185082
2020 1                       189439
     2                       176864
     3                       224264
     4                       202986
2021 1                       201403
     2                       231668
     3                       273794
     4                       265021

In [1]:
net_income = pd.read_csv(Path("Resources/FDICQBPs.csv"))
net_income.head()

NameError: name 'pd' is not defined

In [127]:
net_income = net_income.drop(columns = 'Securities and Other Gains/Losses, Net')
net_income = net_income.loc[net_income['Year']>=2014]
net_income.head()

,Year,Quarter,Net Operating Income
24,2014,1,36.8
25,2014,2,39.8
26,2014,3,38.1
27,2014,4,36.0
28,2015,1,39.0


In [128]:
quarterly_net_income = net_income.groupby(['Year','Quarter']).sum()
quarterly_net_income = quarterly_net_income.rename(columns={'Net Operating Income':'Net Operating Income(Billions)'})
display(quarterly_net_income.head())
display(quarterly_fraud_reporting.head())

Net Operating Income(Billions)
Year Quarter                                
2014 1                                  36.8
     2                                  39.8
     3                                  38.1
     4                                  36.0
2015 1                                  39.0

Fraud Reporting Count
Year Quarter                       
2014 1                        82364
     2                        95516
     3                        94084
     4                        90868
2015 1                        90566

In [129]:
grouped_data = quarterly_fraud_reporting.merge(quarterly_net_income, left_index=True, right_on=['Year','Quarter'])
grouped_data.head()

Fraud Reporting Count  Net Operating Income(Billions)
Year Quarter                                                       
2014 1                        82364                            36.8
     2                        95516                            39.8
     3                        94084                            38.1
     4                        90868                            36.0
2015 1                        90566                            39.0

In [130]:
lagged_data = grouped_data.shift(1)
lagged_data.columns = ['Lagged Fraud Reporting Count', 'Lagged Net Operating Income(Billions)']
grouped_data = pd.concat([grouped_data,lagged_data], axis = 1, join='inner')
grouped_data

Fraud Reporting Count  Net Operating Income(Billions)  \
Year Quarter                                                          
2014 1                        82364                            36.8   
     2                        95516                            39.8   
     3                        94084                            38.1   
     4                        90868                            36.0   
2015 1                        90566                            39.0   
     2                        96365                            42.6   
     3                       101745                            40.0   
     4                       105491                            40.1   
2016 1                       108801                            38.4   
     2                       128668                            42.9   
     3                       126907                            45.1   
     4                       115900                            42.8   
2017 1                       117009                            43.5   
     2                       121347                            47.6   
     3                       125990                            47.3   
     4                       125829                            25.5   
2018 1                       132985                            55.7   
     2                       140769                            60.2   
     3                       152955                            61.9   
     4                       165279                            59.7   
2019 1                       186318                            60.1   
     2                       169526                            61.5   
     3                       177728                            58.2   
     4                       185082                            52.6   
2020 1                       189439                            17.1   
     2                       176864                            16.4   
     3                       224264                            49.3   
     4                       202986                            58.1   
2021 1                       201403                            75.7   
     2                       231668                            69.8   
     3                       273794                            69.3   
     4                       265021                            63.4   

              Lagged Fraud Reporting Count  \
Year Quarter                                 
2014 1                                 NaN   
     2                             82364.0   
     3                             95516.0   
     4                             94084.0   
2015 1                             90868.0   
     2                             90566.0   
     3                             96365.0   
     4                            101745.0   
2016 1                            105491.0   
     2                            108801.0   
     3                            128668.0   
     4                            126907.0   
2017 1                            115900.0   
     2                            117009.0   
     3                            121347.0   
     4                            125990.0   
2018 1                            125829.0   
     2                            132985.0   
     3                            140769.0   
     4                            152955.0   
2019 1                            165279.0   
     2                            186318.0   
     3                            169526.0   
     4                            177728.0   
2020 1                            185082.0   
     2                            189439.0   
     3                            176864.0   
     4                            224264.0   
2021 1                            202986.0   
     2                            201403.0   
     3                            231668.0   
     4                            273794.0   

              Lagged Net Operating

In [132]:
grouped_data.to_csv('Resources/combined_data.csv')